# Code 5
- Combine all Strategy
  - Missing Value Imputation
  - Outlier Startegy
  - Tranformation Strategy

## 1/ Import Libraries

In [1]:
#CodeSection1
from google.colab import drive
drive.mount('/mntDrive') 

Mounted at /mntDrive


In [49]:
#CodeSection2
import pandas as pd
import numpy as np

## 2/ Import Data

In [50]:
#CodeSection3
train = pd.read_csv('/mntDrive/MyDrive/Project MasterMind/1 - Planning Stage/2- Research/6- New Teaching Method/Loan Prediction (Webinar Flow)/Solution Codes/input/train.csv')
test = pd.read_csv('/mntDrive/MyDrive/Project MasterMind/1 - Planning Stage/2- Research/6- New Teaching Method/Loan Prediction (Webinar Flow)/Solution Codes/input/test.csv')

## 3/ Combine both Datasets (train, test)

In [51]:
#CodeSection4
all_data = [train, test]

### Check Combined Data 
- Missing Value
- Data Types

In [52]:
#CodeSection5
# Combine both to check Missing Value
pd.concat([train, test], axis = 0).isnull().sum()

Loan_ID                0
Gender                24
Married                3
Dependents            25
Education              0
Self_Employed         55
ApplicantIncome        2
CoapplicantIncome      0
LoanAmount            27
Loan_Amount_Term      20
Credit_History        79
Property_Area          0
Loan_Status          367
dtype: int64

In [53]:
#CodeSection6
# Check Data types for each variable
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


## 4/ Numerical Processing

### 4.1/ Impute Numerical Variables

In [15]:
#CodeSection7
# Identify all Numerical features
numerical_features = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']

In [54]:
#CodeSection8
# Import SimpleImputer
from sklearn.impute import SimpleImputer

In [55]:
#CodeSection9

#Initiate SimpleImputer
num_impute = SimpleImputer(strategy = 'median')

# Fit SimpleImputer on Numerical Features
num_impute.fit(train[numerical_features])

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='median', verbose=0)

In [56]:
#CodeSection10
# Impute Train and Test
for dataset in all_data:
  dataset[numerical_features] = num_impute.transform(dataset[numerical_features])

### 4.2/ Outlier Strategy

In [57]:
#CodeSection11

# Write a loop to detect and impute Outliers
for num_var in numerical_features:
  Q1 = train[num_var].quantile(0.25)
  Q3 = train[num_var].quantile(0.75)

  IQR = Q3-Q1

  Lower_Whisker = Q1 - 1.5*IQR
  Upper_Whisker = Q3 + 1.5*IQR

  for dataset in all_data:
    dataset[num_var] = dataset[num_var].apply(lambda x : Upper_Whisker if x > Upper_Whisker else x)

### 4.3/ Transformation Strategy

In [58]:
#CodeSection12
# Import StandardScaler
from sklearn.preprocessing import StandardScaler

In [59]:
#CodeSection13

#Initiate StandardScaler
num_scale = StandardScaler()

# Fit StandardScaler
num_scale.fit(dataset[numerical_features])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [60]:
#CodeSection14
# Scale Train and Test
for dataset in all_data:
  dataset[numerical_features] = num_scale.transform(dataset[numerical_features])

## 5/ Impute Categorical Variables

### Impute all Categorical Variables at once

In [61]:
#CodeSection15
# Identify all Categorical Features
categorical_features = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']

In [24]:
#CodeSection16
# Import SimpleImputer
#from sklearn.impute import SimpleImputer

In [62]:
#CodeSection17

#Initiate SimpleImputer
cat_impute = SimpleImputer(strategy = 'most_frequent')

# Fit SimpleImputer
cat_impute.fit(dataset[categorical_features])

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='most_frequent', verbose=0)

In [63]:
#CodeSection18
# Impute Train and Test
for dataset in all_data:
  dataset[categorical_features] = cat_impute.transform(dataset[categorical_features])

## 6/ Now Re Check all Features

In [64]:
#CodeSection19
# Combine both to check Missing Value
pd.concat([train, test], axis = 0).isnull().sum()

Loan_ID                0
Gender                 0
Married                0
Dependents             0
Education              0
Self_Employed          0
ApplicantIncome        0
CoapplicantIncome      0
LoanAmount             0
Loan_Amount_Term       0
Credit_History         0
Property_Area          0
Loan_Status          367
dtype: int64

## 8/ Prepare Data for Model Model

In [65]:
#CodeSection20
# Check First 5 Rows of Test Data
test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,0.609558,-0.899112,-0.466495,0.322842,0.437674,Urban
1,LP001022,Male,Yes,1,Graduate,No,-0.591421,0.051833,-0.140373,0.322842,0.437674,Urban
2,LP001031,Male,Yes,2,Graduate,No,0.282514,0.242022,1.531000,0.322842,0.437674,Urban
3,LP001035,Male,Yes,2,Graduate,No,-0.925733,0.714959,-0.670321,0.322842,0.437674,Urban
4,LP001051,Male,No,0,Not Graduate,No,-0.500576,-0.899112,-1.118738,0.322842,0.437674,Urban


In [66]:
#CodeSection21
# Get X and y Variables for your Dataset
X = train.drop(['Loan_Status','Loan_ID'], axis =  1)
y = train['Loan_Status']

### Create Dummy Variables for  Categorical Variables

In [67]:
#CodeSection22
# Get Dummy Variables
X = pd.get_dummies(X, drop_first = True)  # Keep only n - 1 Variables - Use "drop_first = True"

## 9/ Build Model

In [68]:
#CodeSection23
#Import Logistic Regression Library
from sklearn.linear_model import LogisticRegression

#Initialize LR
clf = LogisticRegression(max_iter = 1000)

In [69]:
#CodeSection24
# Fit Model
model = clf.fit(X, y)

### Check Accuracy of Model on Train Data

In [70]:
#CodeSection25
# Predict on Train Data
y_pred = model.predict(X)

### We can use a Accuracy Function from Metrics
- Check Train Accuracy

In [71]:
#CodeSection26
# Import metrics library
from sklearn.metrics import accuracy_score

In [72]:
#CodeSection27
# get Actual "y" variables (use "y_true" as variable)
y_true = train['Loan_Status']

In [73]:
#CodeSection28
# Print Train Accuracy
print(f" Train Accuracy : {accuracy_score(y_true, y_pred):0.1%}")

 Train Accuracy : 80.9%


## 10/ Predict and Submit

### Predict

In [74]:
#CodeSection29
# Get all the X Variables from the Test Dataset
X_test = test.drop(['Loan_ID'], axis =  1)

In [75]:
#CodeSection30
#Covert to Dummies
X_test = pd.get_dummies(X_test, drop_first = True)

In [40]:
#CodeSection31
# use this code if the next Code shows error
#X_test['Married_Missing']  = 0

In [76]:
#CodeSection32
# Predict on X_test Data ("X_test_prep")
X_test_prep = model.predict(X_test)

### Create Submission File

In [77]:
#CodeSection33
submission = pd.DataFrame({
    'Loan_ID' : test['Loan_ID'],
    'Loan_Status' : X_test_prep
})

### Export Submission File

In [78]:
#CodeSection34
submission.to_csv('/mntDrive/MyDrive/Project MasterMind/1 - Planning Stage/2- Research/6- New Teaching Method/Loan Prediction (Webinar Flow)/Solution Codes/output/O5_Preproccesing_All.csv', index = False)

In [ ]:
# LB Accuracy : 0.7777